# Animal Detection using YOLOv5

In [1]:
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

## Step 1: Import libraries and model

In [2]:
import torch
import cv2
from PIL import Image
import numpy as np

model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\Harman/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-9-21 Python-3.11.8 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


## Step 2: Load the Video File and Detect Animals Frame by Frame

In [3]:
# OpenCV function to process video and detect animals
def detect_animals_in_video(video_path, output_path):
    # Load the video file
    cap = cv2.VideoCapture(video_path)
    
    # Get video dimensions and frame rate
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Create a VideoWriter object to save the output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame from BGR to RGB (required by YOLOv5 model)
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)

        # Perform detection
        results = model(img)
        
        # Convert results to a format suitable for OpenCV
        labels, cord = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]

        # Draw boxes and labels on the frame
        n = len(labels)
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.5:  # Draw boxes for objects with confidence >= 50%
                x1, y1, x2, y2 = int(row[0] * width), int(row[1] * height), int(row[2] * width), int(row[3] * height)
                confidence = row[4]

                # Draw a rectangle around the object
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # Get the label name and confidence score
                label_name = model.names[int(labels[i])]
                label = f'{label_name} {confidence:.2f}'

                # Put the label text above the rectangle
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Write the frame with the detections to the output video
        out.write(frame)

        # Show the frame (optional)
        cv2.imshow('Animal Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release video objects
    cap.release()
    out.release()
    cv2.destroyAllWindows()


# Call the function to detect animals in the input video and save the output video
detect_animals_in_video('../video/input_video.mp4', '../video/output_video.mp4')